# Travel Multi AI Agent 

## Introduction
This notebook demonstrates a multi-agent system for travel planning using Azure AI Agents. 
The system consists of specialized agents (flight, hotel, restaurant, excursion) that work together
to help users plan their travel. The implementation uses two main architectural layers:
1. **Service Layer**: Individual specialized AI agents that provide domain-specific assistance
2. **Orchestration Layer**: The coordination mechanism that manages agent interactions and conversations

# What I'm Trying to Achieve

This project aims to create an orchestrated multi-agent system for comprehensive travel planning. The system uses specialized agents (flight, hotel, restaurant, and excursion) that work together seamlessly to provide a complete travel planning experience. Rather than forcing users to interact with multiple separate assistants, this approach creates a cohesive conversation flow where each agent handles its specialized domain.

## Challenges Faced

While the system is functional, several challenges have emerged in the coordination and orchestration of the specialized agents. These include issues with agent selection logic, template rendering errors, and conversation flow management. Addressing these challenges is crucial to creating a seamless and efficient travel planning experience.


## Blockers: Agent Coordination Challenges

Several challenges have emerged:

1. **Agent Selection Logic Issues**:
   - The current selection strategy sometimes fails to properly parse agent names or select the correct next agent
   - The orchestration prompts need fine-tuning to better understand conversation state

2. **Template Rendering Errors**:
   - Experiencing `TemplateRenderException` errors when trying to pass conversation context between agents
   - Variable handling in templates (using `{{$lastmessage}}`) is inconsistent

3. **Agent Response Processing**:
   - Some agent responses are not being correctly formatted or displayed
   - Need better parsing of response types and handling of multi-turn conversations

4. **Conversation Flow Management**:
   - Agents sometimes repeat information or fail to pick up where the previous agent left off
   - Need better context preservation between agent transitions

5. **Known Issues**
   ```python
   AgentChatException: Failed to select agent: Agent Failure - 
   Strategy unable to select next agent
   ```
   - Currently addressing agent selection and mapping
   - Improving task completion detection

## Desired Outcome

The goal is to create a seamless experience where these specialized agents work together coherently, in an non sequential order, handling complex travel planning tasks that span multiple domains while maintaining conversation context.

In [ ]:
%pip install semantic-kernel[azure] --upgrade

In [ ]:
%pip show semantic-kernel

## Configuration
Before running an Azure AI Agent, modify your .env file to include:

```bash
AZURE_AI_AGENT_PROJECT_CONNECTION_STRING = "<example-connection-string>"
AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME = "<example-model-deployment-name>"
```

The Azure AI Agent will be able to connect to the Azure AI Platform and run the model deployment specified in the .env file.

In [2]:
from azure.identity.aio import DefaultAzureCredential
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
import os
from openai import AsyncOpenAI

from semantic_kernel.contents import ChatHistoryTruncationReducer, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion


# Load environment variables
load_dotenv()

True

## Service Layer Components

These components represent the individual specialized agents that provide domain-specific functionality.
Each agent is responsible for a specific travel-related task:

- **Flight Agent**: Handles flight bookings and inquiries
- **Hotel Agent**: Manages hotel accommodations
- **Restaurant Agent**: Provides restaurant recommendations
- **Excursion Agent**: Offers activity planning assistance

In [3]:
# Define Agent IDs
FLIGHT_AGENT = os.getenv("FLIGHT_AGENT")
HOTEL_AGENT = os.getenv("HOTEL_AGENT")
RESTAURANT_AGENT = os.getenv("RESTAURANT_AGENT")
EXCURSION_AGENT = os.getenv("EXCURSION_AGENT")

## Orchestration Layer Components

The kernel and custom functions below comprise the orchestration layer that coordinates 
agent interactions and manages the overall conversation flow.

In [8]:
def create_kernel() -> Kernel:
    """Creates a Kernel instance with custom OpenAI ChatCompletion service."""
    client = AsyncOpenAI(
        api_key=os.getenv("GITHUB_TOKEN"),
        base_url="https://models.inference.ai.azure.com/"
    )

    kernel = Kernel()
    chat_completion_service = OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini",
        async_client=client,
        service_id="agent",
    )
    kernel.add_service(chat_completion_service)
    return kernel

## Main Application Logic

The main function initializes both service and orchestration layers and sets up the user interaction loop.

In [ ]:
async def main():
    ai_agent_settings = AzureAIAgentSettings.create()

    # Initialize client and agents for specialized services
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(
            credential=creds
        ) as client,
    ):
        # Get agent definitions and create agent instances for each specialized service
        flight_agent = await client.agents.get_agent(assistant_id=FLIGHT_AGENT)
        hotel_agent = await client.agents.get_agent(assistant_id=HOTEL_AGENT)
        restaurant_agent = await client.agents.get_agent(assistant_id=RESTAURANT_AGENT)
        excursion_agent = await client.agents.get_agent(assistant_id=EXCURSION_AGENT)

        termination_agent_def = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name="termination_agent",
        instructions="""You are a specialized termination agent in a travel planning system. Your sole responsibility is to 
            determine when the travel planning conversation has reached a satisfactory conclusion.
            
            Thought: Analyze the conversation history carefully to determine if all the user's travel needs have been addressed. 
            Consider what specific travel aspects (flights, hotels, restaurants, activities) have been discussed and resolved.
            
            Prepare: Review the conversation for the following indicators:
            - Has the user explicitly mentioned they are satisfied or done with planning?
            - Have all relevant travel details been confirmed (dates, locations, preferences)?
            - Has each specialized agent (flight, hotel, restaurant, excursion) completed their respective tasks?
            - Are there any outstanding questions or unresolved issues that need addressing?
            
            Action:
            Signal conversation termination ONLY when:
            1. The user explicitly requests to end the conversation (keywords like "thank you", "that's all", "we're done")
            2. All travel planning aspects have been adequately addressed:
               - Flight details: dates, times, airlines, prices, booking confirmation
               - Hotel accommodations: check-in/out dates, location, room type, booking confirmation
               - Restaurant recommendations: options provided based on preferences/location
               - Excursion/activities: suggestions aligned with travel dates/interests
            3. The user has expressed satisfaction with the overall travel plan
            4. There are no pending questions or requests from either the user or agents
            
            Your response must be exactly "yes" if the conversation should terminate, or "no" if it should continue.""",
        )
        
        termination_agent = AzureAIAgent(
            client=client,
            definition=termination_agent_def
        )
        
        # Create agent instances
        flight_reviewer = AzureAIAgent(
            client=client,
            definition=flight_agent
        )
        hotel_reviewer = AzureAIAgent(
            client=client,
            definition=hotel_agent
        )
        restaurant_reviewer = AzureAIAgent(
            client=client,
            definition=restaurant_agent
        )
        excursion_reviewer = AzureAIAgent(
            client=client,
            definition=excursion_agent
        )

        # Create Semantic Kernel instance for group chat orchestration
        kernel = create_kernel()

        # Define selection function for determining which agent speaks next
        selection_function = KernelFunctionFromPrompt(
            function_name="selection",
            prompt="""
    You are an orchestrator for a multi-agent conversation. Your task is to analyze the latest message and determine if the current conversation should continue with the same agent or move to the next one.

Rules:
1. If the conversation is incomplete (waiting for user input or needs follow-up), keep the same agent.
2. Only switch agents when the current agent has completed its task.
3. The switching order is: flight_reviewer -> hotel_reviewer -> restaurant_reviewer -> excursion_reviewer

Current agent order:
- flight_reviewer (handles flight bookings and inquiries)
- hotel_reviewer (handles hotel bookings)
- restaurant_reviewer (handles restaurant recommendations)
- excursion_reviewer (handles activity planning)

Analyze the latest message and respond with ONLY the agent name that should speak next.
Latest Message:
{{$lastmessage}}
""",
        )

        # Define termination conditions
        termination_keyword = "exit"
        termination_function = KernelFunctionFromPrompt(
            function_name="termination",
            prompt="""
    Examine the RESPONSE and determine whether the content has been deemed satisfactory.
    If the content is satisfactory, respond with a single word without explanation: yes.
    If specific suggestions are being provided, it is not satisfactory.
    If no correction is suggested, it is satisfactory.
    RESPONSE:
    {{$lastmessage}}
    """,
        )

        # Configure history reducer to limit context size
        history_reducer = ChatHistoryTruncationReducer(target_count=1)

        # Set up the agent group chat with selection and termination strategies
        chat = AgentGroupChat(
            agents=[flight_reviewer, hotel_reviewer,
                    restaurant_reviewer, excursion_reviewer],
            selection_strategy=KernelFunctionSelectionStrategy(
                initial_agent=flight_reviewer,
                function=selection_function,
                kernel=kernel,
                history_variable_name="lastmessage",
                history_reducer=history_reducer,
                result_parser=parse_agent_name  # Add the parser function
            ),
            termination_strategy=KernelFunctionTerminationStrategy(
                # Use the dedicated termination agent for deciding when to end the conversation
                agents=[termination_agent],
                function=termination_function,
                kernel=kernel,
                result_parser=lambda result: str(result.value).lower().strip() == "yes" if result and hasattr(result, 'value') else False,
                history_variable_name="lastmessage",
                maximum_iterations=10,
                history_reducer=history_reducer,
            ),
        )

        print("Ready! Type your input (or 'exit' to quit, 'reset' to restart).")

        # Main interaction loop
        while True:
            # user_input = input("User > ").strip()
            user_input = "flight details from delhi to bengaluru for 20th March 2025"
            if not user_input:
                continue
            if user_input.lower() == "exit":
                break
            if user_input.lower() == "reset":
                await chat.reset()
                print("[Conversation has been reset]")
                continue

            # Add user input to the group chat context
            await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
            print(f"User > {user_input}")

            # Invoke the group chat to get responses from agents
            async for response in chat.invoke():
                agent_name = response.name if hasattr(
                    response, 'name') else '*'
                print(f"# {agent_name}: '{response.content}'")
            chat.is_complete = False

# Run the main function in the notebook
await main()

Ready! Type your input (or 'exit' to quit, 'reset' to restart).
User > flight details from delhi to bengaluru for 20th March 2025
# Flight_Agent: 'Hello! I'm here to assist you with your flight-related queries. It seems like you're looking to **search for flights** from Delhi to Bengaluru for the 20th of March, 2025. Could you confirm if this is a one-way trip or if you'd like to include a return date? If it's a round trip, please provide the return date as well!'


Failed to select agent: Agent Failure - Strategy unable to select next agent: arguments=None description=None id='asst_NmyVpGdP2K3gDbpRsZT1Zagt' instructions='You are FlightAgent, a virtual assistant specialized in handling flight-related queries. Your role includes assisting users with searching for flights, retrieving flight details, checking seat availability, and providing real-time flight status. Follow the instructions below to ensure clarity and effectiveness in your responses:\n\n### Task Instructions:\n1. **Recognizing Intent**:\n   - Identify the user\'s intent based on their request, focusing on one of the following categories:\n     - Searching for flights\n     - Retrieving flight details using a flight ID\n     - Checking seat availability for a specified flight\n     - Providing real-time flight status using a flight number\n   - If the intent is unclear, politely ask users to clarify or provide more details.\n\n2. **Processing Requests**:\n   - Depending on the identifi

AgentChatException: Failed to select agent